In [1]:
import numpy as np 
import pandas as pd
from os import path
from random import choice, shuffle

from ga4stpg.graph import Graph, ReaderORLibrary
from ga4stpg.graph.steiner import (prunning_mst, shortest_path,
                            shortest_path_origin_prim,
                            prunning_kruskal_mst,
                            shortest_path_with_origin)

from ga4stpg.graph.util import (is_steiner_tree, 
                                has_cycle,
                                gg_total_weight)

from ga4stpg.tree.prunningcrossover import PrunningMSTCrossover
from ga4stpg.tree.evaluation import EvaluateTreeGraph

In [2]:
datasets = path.join('..', '..', 'ppgi-stpg-gpx', 'datasets', 'ORLibrary')
instance_file = 'steinc5.txt'

path_to_instance = path.join(datasets, instance_file)

assert path.exists(path_to_instance)

stpg = ReaderORLibrary().parser(path_to_instance)

In [3]:

crossover = PrunningMSTCrossover(stpg)
evaluator = EvaluateTreeGraph(stpg)

In [4]:


vertices = list(stpg.graph.vertices)

shuffle(vertices)

print(vertices[:10],' ...')

[141, 407, 235, 227, 219, 155, 22, 484, 206, 262]  ...


In [5]:
# Exemplo para criação de um único individuo

# v = vertices.pop()

# aa, aa_cost = shortest_path_with_origin(stpg.graph, v, stpg.terminals)

# r, test = is_steiner_tree(aa, stpg)

# (r, test)

In [6]:
%%time

data = list()

for _ in range(200):
    v = vertices.pop()
    aa, aa_cost = prunning_mst(stpg.graph, v, stpg.terminals)
    e_aacost, _ = evaluator(aa)
    assert aa_cost == e_aacost, f'{aa_cost} != {e_aacost}'
    is_aa_st, _ = is_steiner_tree(aa, stpg)
    
    u = vertices.pop()
    bb, bb_cost = prunning_mst(stpg.graph, u, stpg.terminals)
    e_bbcost, _ = evaluator(bb)
    assert bb_cost == e_bbcost, f'{bb_cost} != {e_bbcost}'
    is_bb_st, _ = is_steiner_tree(bb, stpg)
    
    child = crossover(aa, bb)
    cc_cost, _  = evaluator(child)
    is_cc_st, _ = is_steiner_tree(child, stpg)

    equal_to_aa = child.edges == aa.edges
    equal_to_bb = child.edges == bb.edges
    
    data.append([v, u, is_aa_st, is_bb_st, is_cc_st, aa_cost, bb_cost, cc_cost, equal_to_aa, equal_to_bb])

Wall time: 12.6 s


In [7]:
assert aa.edges == aa.edges

In [8]:
columns_name = ['start_a', 
                'start_b', 
                'is_aa_st', 
                'is_bb_st', 
                'is_cc_st' ,
                'a_cost', 
                'b_cost', 
                'c_cost',
                'equal_to_aa',
                'equal_to_bb'
                ]
                
frame = pd.DataFrame(data, columns=columns_name)

In [9]:
frame['less_both'] = (frame['c_cost'] < frame['a_cost']) & (frame['c_cost'] < frame['b_cost'])

In [10]:
frame['less_equal_both'] = (frame['c_cost'] <= frame['a_cost']) & (frame['c_cost'] <= frame['b_cost'])

In [11]:
frame['less_a'] = (frame['c_cost'] <= frame['a_cost'])

In [12]:
frame['less_b'] = (frame['c_cost'] <= frame['b_cost'])

In [13]:
frame['greater_a'] = (frame['c_cost'] > frame['a_cost'])
frame['greater_b'] = (frame['c_cost'] > frame['b_cost'])

In [14]:
frame.head()

,start_a,start_b,is_aa_st,is_bb_st,is_cc_st,a_cost,b_cost,c_cost,equal_to_aa,equal_to_bb,less_both,less_equal_both,less_a,less_b,greater_a,greater_b
0,412,352,True,True,True,1630,1637,1630,False,False,False,True,True,True,False,False
1,131,318,True,True,True,1637,1640,1634,False,False,True,True,True,True,False,False
2,493,299,True,True,True,1634,1637,1634,False,False,False,True,True,True,False,False
3,279,40,True,True,True,1636,1636,1636,False,False,False,True,True,True,False,False
4,153,327,True,True,True,1640,1640,1640,False,False,False,True,True,True,False,False


In [15]:
frame[['is_aa_st', 'is_bb_st', 'is_cc_st', 'equal_to_bb', 'equal_to_aa']].describe()

,is_aa_st,is_bb_st,is_cc_st,equal_to_bb,equal_to_aa
count,200,200,200,200,200
unique,1,1,1,1,1
top,True,True,True,False,False
freq,200,200,200,200,200


In [16]:
frame[['a_cost', 'b_cost','c_cost']].min()

a_cost    1630
b_cost    1630
c_cost    1630
dtype: int64

In [17]:
frame[['a_cost', 'b_cost','c_cost']].max()

a_cost    1655
b_cost    1652
c_cost    1652
dtype: int64

In [18]:
frame[['less_both', 'less_a', 'less_b', 'less_equal_both']].describe()

,less_both,less_a,less_b,less_equal_both
count,200,200,200,200
unique,2,2,2,2
top,False,True,True,False
freq,186,148,132,112


In [19]:
frame[frame['greater_a'] & frame['greater_b']][['a_cost', 'b_cost', 'c_cost', 'equal_to_bb', 'equal_to_aa']]

,a_cost,b_cost,c_cost,equal_to_bb,equal_to_aa
8,1637,1638,1639,False,False
16,1640,1636,1641,False,False
31,1630,1634,1636,False,False
72,1638,1634,1640,False,False
124,1638,1637,1640,False,False
127,1638,1636,1640,False,False
161,1638,1637,1639,False,False
181,1637,1638,1639,False,False


In [20]:
frame[frame['greater_a'] & frame['less_b'] ][['a_cost', 'b_cost', 'c_cost', 'equal_to_bb', 'equal_to_aa']]

,a_cost,b_cost,c_cost,equal_to_bb,equal_to_aa
7,1630,1636,1636,False,False
24,1637,1652,1652,False,False
26,1634,1638,1638,False,False
32,1630,1640,1640,False,False
33,1630,1634,1634,False,False
37,1630,1637,1634,False,False
40,1636,1640,1640,False,False
41,1634,1637,1636,False,False
49,1630,1638,1638,False,False
56,1633,1636,1636,False,False


In [22]:
frame[['a_cost', 'b_cost', 'c_cost']].corr()

,a_cost,b_cost,c_cost
a_cost,1.000000,-0.035680,0.434085
b_cost,-0.035680,1.000000,0.496839
c_cost,0.434085,0.496839,1.000000
